<a href="https://colab.research.google.com/github/Dih08/projetos-simples-e-aleatorios/blob/main/Modelo_em_produ%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
reg = np.random.RandomState(0)
X, y = make_classification(n_samples=1000, n_features=5,
                           random_state=reg)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Ajustando um Random Florest

In [ ]:
clf = RandomForestClassifier(n_estimators=10, random_state=reg)
model = clf.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, recall_score, precision_score
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       160
           1       0.96      0.98      0.97       170

    accuracy                           0.97       330
   macro avg       0.97      0.97      0.97       330
weighted avg       0.97      0.97      0.97       330



In [ ]:
model_metrics = dict()
model_metrics['recall'] = recall_score(y_test, pred)
model_metrics['precision'] = precision_score(y_test, pred)

In [ ]:
model_metrics

{'recall': 0.9764705882352941, 'precision': 0.9595375722543352}

In [ ]:
import json
with open('model_metrics.json', 'w') as fp:
    json.dump(model_metrics, fp)

In [ ]:
#Salvando o mdelo

from joblib import dump, load
dump(model, 'model.joblib')



['model.joblib']

Criando API

In [ ]:
from flask import Flask, request, jsonify, url_for, render_template,redirect
import pickle
import joblib
import traceback
import pandas as pd
import numpy as np

In [ ]:
app = Flask(__name__)

In [ ]:
modeal = joblib.load('model.joblib')

In [ ]:
app.route('/')
def home():
    return render_template('index.html')

In [ ]:
app.route('/predict', methods=['POST', 'GET'])
def predict():
  features = [x for x in request.form.values()]

  features_array = np.array(features)
  pandas_features = pd.DataFrame([features_array])
  print(features)

  prediction = model.predict(pandas_features)
  print(prediction)
  preds = int(prediction[0])
  return render_template('index.html', prediction_text='{}'.format(preds))

In [ ]:
app.route('/model_health/<model_id>', methods=['GET'])
def model_healt(model_id):
  with open('model_metrics.json'.format(model_id)) as fp:
    model_metrics = json.load(fp)
    return model_metrics

if __name__ == '__main__':
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
